In [ ]:
!pip install tensorflow tensorflow-io matplotlib xgboost catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import itertools
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
import pywt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

#listing gunshot n non gunshot files
POS = '/content/drive/MyDrive/aidataset/gunshot'
NEG = '/content/drive/MyDrive/aidataset/nongunshot'
pos = tf.data.Dataset.list_files(POS+'/*.wav')
neg = tf.data.Dataset.list_files(NEG+'/*.wav')

positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
#calculating the quotient n remainder
repeat_count = len(negatives) // len(positives)
remainder = len(negatives) % len(positives)
#oversampling the minority with given values of quotient n remainder
new_positives = positives.repeat(repeat_count) #repeat that many times as quotient
new_positives = new_positives.concatenate(positives.take(remainder)) #add that many as remainder
data = new_positives.concatenate(negatives) #concatenate gunshot n nonshot

# function provided in tenserflow
def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    wav = wav[:40000]
    zero_padding = tf.zeros([40000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav], 0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram_rgb = tf.stack([spectrogram] * 3, axis=-1) #converting grayscale to rgb
    return spectrogram_rgb, label

data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=5000)
data = data.batch(16)
data = data.prefetch(8)

train = data.take(130)
test = data.skip(130).take(30)
ttest = data.skip(160).take(20)

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(1241, 257, 1)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer with dropout rate of 0.5
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer with dropout rate of 0.5

# Create a feature extraction model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-1].output)

# Extract features for all samples in the dataset
features = []
labels = []
for spectrogram, label in train:
    extracted_features = feature_extractor.predict(spectrogram)
    features.append(extracted_features)
    labels.append(label)

features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)

# Extract features from test set
test_features = []
test_labels = []
for spectrogram, label in test:
    extracted_features = feature_extractor.predict(spectrogram)
    test_features.append(extracted_features)
    test_labels.append(label)

test_features = np.concatenate(test_features, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Define classifiers
classifiers = {
    "KNN (k=2)": KNeighborsClassifier(n_neighbors=2),
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "SVM": SVC(),
    "Random Forests": RandomForestClassifier(),
    "Naive Bayes": GaussianNB(),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Decision Trees": DecisionTreeClassifier(),
    "CatBoost": CatBoostClassifier(verbose=False),
    "XGBoost": XGBClassifier(use_label_encoder=False, verbosity=0),
    "adaBoost": AdaBoostClassifier(),
}

# Initialize dictionaries to store results
accuracy_results = {}
precision_results = {}
recall_results = {}
f1_results = {}
confusion_matrices = {}
train_accuracy_results = {}
test_accuracy_results = {}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(features, labels)
    y_pred = clf.predict(test_features)
    y_train_pred = clf.predict(features)

    # Compute evaluation metrics
    accuracy = accuracy_score(test_labels, y_pred)
    precision = precision_score(test_labels, y_pred)
    recall = recall_score(test_labels, y_pred)
    f1 = f1_score(test_labels, y_pred)
    confusion = confusion_matrix(test_labels, y_pred)
    train_accuracy = accuracy_score(labels, y_train_pred)

    # Storing results in dictionaries
    accuracy_results[name] = accuracy
    precision_results[name] = precision
    recall_results[name] = recall
    f1_results[name] = f1
    confusion_matrices[name] = confusion
    train_accuracy_results[name] = train_accuracy

# Ensemble methods (Voting Classifier)
voting_clf = VotingClassifier(estimators=list(classifiers.items()), voting='hard')
voting_clf.fit(features, labels)
y_pred_voting = voting_clf.predict(test_features)
y_train_pred_voting = voting_clf.predict(features)

# Compute evaluation metrics for Voting Classifier
accuracy_results['Voting Classifier'] = accuracy_score(test_labels, y_pred_voting)
precision_results['Voting Classifier'] = precision_score(test_labels, y_pred_voting)
recall_results['Voting Classifier'] = recall_score(test_labels, y_pred_voting)
f1_results['Voting Classifier'] = f1_score(test_labels, y_pred_voting)
confusion_matrices['Voting Classifier'] = confusion_matrix(test_labels, y_pred_voting)
train_accuracy_results['Voting Classifier'] = accuracy_score(labels, y_train_pred_voting)

# Print results
for name, result in accuracy_results.items():
    print(f"Classifier: {name}")
    print(f"Test Accuracy: {result}")
    print(f"Train Accuracy: {train_accuracy_results[name]}")
    print(f"Precision: {precision_results[name]}")
    print(f"Recall: {recall_results[name]}")
    print(f"F1 Score: {f1_results[name]}")
    print(f"Confusion Matrix:\n{confusion_matrices[name]}\n")

# Plot training and test accuracies
plt.figure(figsize=(10, 6))
plt.plot(train_accuracy_results.keys(), train_accuracy_results.values(), label='Training Accuracy', marker='o')
plt.plot(accuracy_results.keys(), accuracy_results.values(), label='Test Accuracy', marker='o')
plt.xlabel('Classifier')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy of Different Classifiers')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Plot precision, recall, and F1-score
plt.figure(figsize=(10, 6))
plt.plot(precision_results.keys(), precision_results.values(), label='Precision')
plt.plot(recall_results.keys(), recall_results.values(), label='Recall')
plt.plot(f1_results.keys(), f1_results.values(), label='F1-score')
plt.xlabel('Classifier')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1-score of Different Classifiers')
plt.xticks(rotation=45)
plt.legend()
plt.show()
